In [1]:
# Import necessary libraries
import os
import sys
import json
import torch
import bigbench

# Import modules from the codebase
from transformer_confidence.data_loader.bigbench_loader import BigBenchLoader
from transformer_confidence.prompt_generator.prompt_builder import PromptBuilder
from transformer_confidence.prompt_generator.prompt_templates import (
    FIRST_ORDER_PROMPT,
    SECOND_ORDER_PROMPT_2AFC,
    SECOND_ORDER_PROMPT_CONTINUOUS,
    SECOND_ORDER_PROMPT_INTUITIVE
)
from transformer_confidence.nndif_client.nndif_api import NNDIFClient
from transformer_confidence.utils.storage import StorageManager

# Import NNsight and configure API key
import nnsight
from nnsight import CONFIG

/Users/adamimos/anaconda3/envs/transformer_confidence_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'load_metric' from 'datasets' (/Users/adamimos/anaconda3/envs/transformer_confidence_env/lib/python3.9/site-packages/datasets/__init__.py)

In [8]:
import requests
import base64
import json

def fetch_bigbench_task_jsons(task_name, ref='main', token=None):
    """
    Fetches all JSON files from a specific BigBench task folder and its subfolders.

    Parameters:
        task_name (str): Name of the task folder in bigbench/benchmark_tasks
        ref (str): The name of the commit/branch/tag. Default is 'main'.
        token (str): GitHub Personal Access Token for authenticated requests (optional)

    Returns:
        dict: A dictionary where keys are file paths and values are parsed JSON contents
    """
    base_url = 'https://api.github.com/repos/google/BIG-bench/contents'
    task_path = f'bigbench/benchmark_tasks/{task_name}'
    headers = {'Accept': 'application/vnd.github.v3+json'}
    if token:
        headers['Authorization'] = f'token {token}'

    def fetch_contents(path):
        url = f'{base_url}/{path}?ref={ref}'
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            raise Exception(f'Error fetching contents: {response.status_code} {response.reason}')
        return response.json()

    def process_item(item):
        if item['type'] == 'file' and item['name'].endswith('.json'):
            file_content = requests.get(item['download_url'], headers=headers).content
            return {item['path']: json.loads(file_content.decode('utf-8'))}
        elif item['type'] == 'dir':
            return process_folder(item['path'])
        return {}

    def process_folder(folder_path):
        contents = fetch_contents(folder_path)
        result = {}
        for item in contents:
            result.update(process_item(item))
        return result

    return process_folder(task_path)


In [9]:

# Example usage:
owner = 'google'
repo = 'BIG-bench'
task_name = 'simple_arithmetic'
filepath = f'bigbench/benchmark_tasks/{task_name}/task.json'
ref = 'main'  # or specify a commit hash or branch name

# If you have a GitHub token, you can provide it here to increase rate limits
token = None  # Replace with your token if available

task_data = fetch_bigbench_task_jsons(task_name, ref, token)

print(task_data)


{'bigbench/benchmark_tasks/simple_arithmetic/results/scores_BIG-G_125m_T=0.json': {'model': {'additional_details': 'No additional information about training or evaluation protocol for this model.', 'decoding_params': {'temperature': 0}, 'description': 'A dense decoder-only transformer architecture, similar to that in LaMDA (https://arxiv.org/abs/2201.08239). See the BIG-bench paper for details.', 'flop_matched_non_embedding_params': 134228992, 'model_family': 'BIG-G T=0', 'model_name': '125m', 'non_embedding_params': 134228992, 'total_params': 166996992, 'training_batch_size': 262144, 'training_steps': 649200}, 'scores': [{'high_score': 1.0, 'low_score': 0.0, 'number_of_shots': 0, 'preferred_score': 'exact_str_match', 'score_dict': {'exact_str_match': 0.11}, 'subtask_description': 'Exact string match arithmetic on 1 digits.'}, {'high_score': 1.0, 'low_score': 0.0, 'number_of_shots': 0, 'preferred_score': 'exact_str_match', 'score_dict': {'exact_str_match': 0.0}, 'subtask_description': 